# Этот ноутбук необходим для оценки показателей модели, обученной на Kaggle, так как Kaggle сломался и время на эксплуатацию GPU кончилось

In [23]:
%pip install clean-fid

Note: you may need to restart the kernel to use updated packages.


In [24]:
# Установка необходимых библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from tqdm import tqdm
import tempfile
from PIL import Image

In [25]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim=128, channels_img=3, features_g=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        
        self.initial = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g*16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g*16),
            nn.ReLU(True)
        )
        
        # Добавляем residual blocks для лучшего качества
        self.res_blocks = nn.Sequential(
            ResidualBlock(features_g*16),
            ResidualBlock(features_g*16),
        )
        
        self.main = nn.Sequential(
            # 4x4 -> 8x8
            nn.ConvTranspose2d(features_g*16, features_g*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g*8),
            nn.ReLU(True),
            
            # 8x8 -> 16x16
            nn.ConvTranspose2d(features_g*8, features_g*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g*4),
            nn.ReLU(True),
            
            # Self-Attention layer для глобальной согласованности
            SelfAttention(features_g*4),
            
            # 16x16 -> 32x32
            nn.ConvTranspose2d(features_g*4, features_g*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g*2),
            nn.ReLU(True),
            
            # 32x32 -> 64x64
            nn.ConvTranspose2d(features_g*2, channels_img, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, z):
        z = z.view(-1, self.z_dim, 1, 1)
        x = self.initial(z)
        x = self.res_blocks(x)
        return self.main(x)

class Critic(nn.Module):
    def __init__(self, channels_img=3, features_d=128):
        super(Critic, self).__init__()
        self.main = nn.Sequential(
            # 64x64 -> 32x32
            nn.Conv2d(channels_img, features_d, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 32x32 -> 16x16
            nn.Conv2d(features_d, features_d*2, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(features_d*2),  # Заменяем BatchNorm на InstanceNorm
            nn.LeakyReLU(0.2, inplace=True),
            
            # 16x16 -> 8x8
            nn.Conv2d(features_d*2, features_d*4, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(features_d*4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Self-Attention layer
            SelfAttention(features_d*4),
            
            # 8x8 -> 4x4
            nn.Conv2d(features_d*4, features_d*8, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(features_d*8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 4x4 -> 1x1
            nn.Conv2d(features_d*8, 1, 4, 1, 0, bias=False),
        )
    
    def forward(self, x):
        return self.main(x).view(x.size(0), -1)

# Дополнительные модули
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(channels),
            nn.ReLU(True),
            nn.Conv2d(channels, channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(channels),
        )
    
    def forward(self, x):
        return x + self.block(x)

class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Conv2d(in_dim, in_dim//8, 1)
        self.key = nn.Conv2d(in_dim, in_dim//8, 1)
        self.value = nn.Conv2d(in_dim, in_dim, 1)
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x):
        batch, C, width, height = x.size()
        query = self.query(x).view(batch, -1, width*height).permute(0, 2, 1)
        key = self.key(x).view(batch, -1, width*height)
        energy = torch.bmm(query, key)
        attention = self.softmax(energy)
        value = self.value(x).view(batch, -1, width*height)
        out = torch.bmm(value, attention.permute(0, 2, 1))
        out = out.view(batch, C, width, height)
        return self.gamma * out + x

In [26]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [27]:
# Инициализация моделей
generator = Generator().to(device)
critic = Critic().to(device)

In [28]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Определим преобразования для изображений
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Изменяем размер до 64x64
    transforms.ToTensor(),  # Преобразуем в тензор
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Нормализуем в диапазон [-1, 1]
])

# Создаем кастомный датасет
class AnimeFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Директория с изображениями.
            transform (callable, optional): Трансформации для изображений.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        
        # Рекурсивно собираем все изображения
        for filename in os.listdir(root_dir):
            if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                self.image_paths.append(os.path.join(root_dir, filename))
        
        print(f"Найдено {len(self.image_paths)} изображений")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Возвращаем изображение и метку (для совместимости, но метки не используются)
        return image, 0

# Создаем датасет
dataset_path = './data_from_learn/input/images'  # Путь к вашим изображениям
anime_dataset = AnimeFaceDataset(root_dir=dataset_path, transform=transform)

# Создаем DataLoader
batch_size = 64
dataloader = DataLoader(
    anime_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,  # Количество процессов для загрузки данных
    # pin_memory=True  # Ускоряет передачу данных на GPU
)

print(f"DataLoader создан. Размер батча: {batch_size}")
print(f"Количество батчей: {len(dataloader)}")

Найдено 63565 изображений
DataLoader создан. Размер батча: 64
Количество батчей: 994


In [29]:
import tempfile
import os
from PIL import Image
from tqdm import tqdm
from cleanfid import fid

def get_images_to_fid_format(images_list):
    """
    Преобразование картинок для clean-fid формата
    """
    all_images = torch.cat(images_list, dim=0)
    all_images = all_images * 0.5 + 0.5
    all_images = all_images.mul(255).add(0.5).clamp(0, 255).permute(0, 2, 3, 1).to('cpu', torch.uint8).numpy()
    return all_images

def save_images_to_folder(image_array, folder_path, max_to_save=None):
    """
    Сохранение изображений
    """
    os.makedirs(folder_path, exist_ok=True)

    num_images = image_array.shape[0]
    if max_to_save is not None and max_to_save < num_images:
        images_to_process = image_array[:max_to_save]
        print(f"Сохраняем первые {max_to_save} изображений в {folder_path}...")
    else:
        images_to_process = image_array
        print(f"Сохраняем все {num_images} изображений в {folder_path}...")

    for i in tqdm(range(images_to_process.shape[0]), desc=f"Сохранение в {os.path.basename(folder_path)}"):
        img = Image.fromarray(images_to_process[i])
        img.save(os.path.join(folder_path, f"{i:05d}.png"))

def calculate_final_fid_fixed(
    trained_gen,
    dataloader,
    z_dim,
    device,
    num_samples=10000,
    save_generated_path=None,
    save_real_path=None,
    max_saved_images=100
):
    """
    Собирает реальные и фейковые изображения, преобразует их,
    сохраняет во временные папки для FID и опционально сохраняет
    изображения в постоянные папки.
    """

    # 1. Сбор реальных изображений
    print("Собираем реальные изображения...")
    real_images_list = []
    for images, _ in dataloader:
        real_images_list.append(images.cpu())
        if len(real_images_list) * dataloader.batch_size >= num_samples:
            break
    real_images_array = get_images_to_fid_format(real_images_list)[:num_samples]

    if save_real_path:
        print(f"\nОбнаружен save_real_path. Сохраняем реальные изображения в: {save_real_path}")
        # Сохраняем только часть, определенную max_saved_images
        save_images_to_folder(real_images_array, save_real_path, max_to_save=max_saved_images)
        print("Сохранение реальных изображений завершено.")

    # 2. Генерация фейковых изображений
    print(f"\nГенерируем {num_samples} фейковых изображений...")
    trained_gen.eval()
    fake_images_list = []
    batch_size = dataloader.batch_size

    for i in tqdm(range(0, num_samples, batch_size), desc="Генерация изображений"):
        current_batch_size = min(batch_size, num_samples - i)
        noise = torch.randn(current_batch_size, z_dim, 1, 1, device=device)
        with torch.no_grad():
            fake_batch = trained_gen(noise).cpu()
        fake_images_list.append(fake_batch)

    fake_images_array = get_images_to_fid_format(fake_images_list)

    if save_generated_path:
        print(f"\nОбнаружен save_generated_path. Сохраняем сгенерированные изображения в: {save_generated_path}")
        # Сохраняем только часть, определенную max_saved_images
        save_images_to_folder(fake_images_array, save_generated_path, max_to_save=max_saved_images)
        print("Сохранение сгенерированных изображений завершено.")

    # 3. Расчет FID: ИСПОЛЬЗУЕМ ВРЕМЕННЫЕ ПАПКИ
    print("\nНачинаем расчет FID...")

    with tempfile.TemporaryDirectory() as real_dir, tempfile.TemporaryDirectory() as fake_dir:

        # Для FID сохраняем все необходимые num_samples изображений
        print("Сохраняем реальные изображения во временную папку для FID...")
        save_images_to_folder(real_images_array, real_dir, max_to_save=None)

        print("Сохраняем фейковые изображения во временную папку для FID...")
        save_images_to_folder(fake_images_array, fake_dir, max_to_save=None)

        fid_value = fid.compute_fid(
            real_dir,
            fake_dir,
            model_name="inception_v3",
            device=device,
            verbose=True,
            num_workers=0
        )
        return fid_value

In [30]:
state_dict = torch.load('./data_from_learn/weights/netG_epoch_020.pth', map_location=device)
generator.load_state_dict(state_dict)
generator.eval()

Generator(
  (initial): Sequential(
    (0): ConvTranspose2d(128, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (res_blocks): Sequential(
    (0): ResidualBlock(
      (block): Sequential(
        (0): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResidualBlock(
      (block): Sequential(
        (0): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2):

In [31]:
# Вычисление FID
final_fid_score = calculate_final_fid_fixed(
    trained_gen=generator,
    dataloader=dataloader,
    z_dim=128,
    device=device,
    num_samples=10000,
    save_generated_path="generated_samples",
    save_real_path="real_samples",
    max_saved_images=100
)

print(f"\nФинальный FID score: {final_fid_score:.2f}")

Собираем реальные изображения...

Обнаружен save_real_path. Сохраняем реальные изображения в: real_samples
Сохраняем первые 100 изображений в real_samples...


Сохранение в real_samples: 100%|██████████| 100/100 [00:00<00:00, 1263.86it/s]


Сохранение реальных изображений завершено.

Генерируем 10000 фейковых изображений...


Генерация изображений: 100%|██████████| 157/157 [01:27<00:00,  1.80it/s]



Обнаружен save_generated_path. Сохраняем сгенерированные изображения в: generated_samples
Сохраняем первые 100 изображений в generated_samples...


...

compute FID between two folders
Found 20000 images in the folder C:\Users\Boris\AppData\Local\Temp\tmp5920kqb9


FID tmp5920kqb9 : 100%|██████████| 625/625 [15:37<00:00,  1.50s/it]


Found 20000 images in the folder C:\Users\Boris\AppData\Local\Temp\tmp__auefw6


FID tmp__auefw6 : 100%|██████████| 625/625 [15:35<00:00,  1.50s/it]



Финальный FID score: 71.12
